In [111]:
import struct

class IndexStorage:
    encoding = 'utf8'
    
    len_format = 'B'
    len_size = 1
    int_format = 'I'
    int_size = 4
    
    @staticmethod
    def _round_size(sz):
        if sz % 4:
            sz += 4 - (sz % 4)
            
        return sz
    
    @classmethod
    def _calc_str_size(cls, l):
        return cls._round_size(cls.len_size + l)
    
    @classmethod
    def _calc_lst_size(cls, l):
        return cls._round_size(cls.len_size + l * cls.int_size)
    
    @classmethod
    def _str_struct(cls, l): 
        if l > 256 ** cls.len_size:
            raise Exception("[IndexStorage._str_struct] Too long string(len={})".format(l))
        return struct.Struct('{}{}s'.format(cls.len_format, l))
    
    @classmethod
    def _lst_struct(cls, l): 
        if l > 256 ** cls.len_size:
            raise Exception("[IndexStorage._lst_struct] Too long list(len={})".format(l))
        return struct.Struct('{}{}'.format(cls.len_format, cls.int_format * l))
    
    @classmethod
    def str2byte(cls, s):
        try:
            b = s.encode(cls.encoding)
        except:
            raise Exception("[IndexStorage.str2byte] Cannot encode string(s='{}')".format(s))
            
        return cls._str_struct(len(s)).pack(len(s), b)
    
    @classmethod
    def byte2str(cls, b):
        try:
            l = struct.unpack(cls.len_format, b[:cls.len_size])[0]
        except:
            raise Exception("[IndexStorage.byte2str] Cannot extract string length(b={})".format(b))
        
        try:
            _l, s = cls._str_struct(l).unpack(b[:cls._calc_str_size(l)])
        except:
            raise Exception("[IndexStorage.byte2str] Cannot unpack string(b={})".format(b))
        
        try:
            s = s.decode(cls.encoding)
        except:
            raise Exception("[IndexStorage.byte2str] Cannot decode string(s='{}')".format(s))
            
        return s
    
    @classmethod
    def lst2byte(cls, lst):
        l = len(lst)
        return cls._lst_struct(l).pack(l, *lst)        
    
    @classmethod
    def byte2lst(cls, b):
        try:
            l = struct.unpack(cls.len_format, b[:cls.len_size])[0]
            
        except:
            raise Exception("[IndexStorage.byte2lst] Cannot extract list length(b={})".format(b))
            
        try:
            t = cls._lst_struct(l).unpack(b[:cls._calc_lst_size(l)])
        except:
            raise Exception("[IndexStorage.byte2lst] Cannot unpack list(b={})".format(b))            
        
        return list(t)[1:]
    
    @classmethod
    def test(cls):
        test_str = 'adf' * 40
        b = IndexStorage.str2byte(test_str)
        s = IndexStorage.byte2str(b)
        if test_str != s:
            raise Exception("[IndexStrorage] String coding-encoding error.")
    
        test_lst = [1, 1231231231, 1000] * 40
        b = IndexStorage.lst2byte(test_lst)
        l = IndexStorage.byte2lst(b)
        if test_lst != l:
            raise Exception("[IndexStrorage] List coding-encoding error.")
            
IndexStorage.test()